In [40]:
#!/usr/bin/python
# -*- coding: latin-1 -*-
"""This notebook builds the CSV-like file describing the contributions to WLE in Spain in YEAR"""

import os, sys, inspect
from datetime import datetime, timedelta

current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
folder_parts = current_folder.split(os.sep)
pywikibot_folder = os.sep.join(folder_parts[0:-1])

if current_folder not in sys.path:
    sys.path.insert(0, current_folder)
if pywikibot_folder not in sys.path:
    sys.path.insert(0, pywikibot_folder)

import pywikibot as pb
from pywikibot import pagegenerators, textlib
from StringIO import StringIO
import mwparserfromhell as mwh
import pandas as pd
import numpy as np
import re

In [45]:
WLE_CATEGORY = u"Category:Images from Wiki Loves Earth {0} in Spain".format(YEAR)
commons_site = pb.Site("commons", "commons")

In [48]:
YEAR                = 2015
BASE_SCI_DB_NAME    = u"Commons:Wiki Loves Earth 2016 in Spain"
SCI_DB_PAGE         = BASE_SCI_DB_NAME + u"/SCI DB"

BASE_WLE_NAME       = u"Commons:Wiki Loves Earth {0} in Spain".format(YEAR)
LOG_PAGE            = BASE_WLE_NAME + u"/Log"

start_datetime =  datetime(YEAR, 6, 1)

In [49]:
BASE_WLE_NAME

u'Commons:Wiki Loves Earth 2015 in Spain'

In [50]:
communities = {
    'ES-AN': u'Andalusia',
    'ES-AR': u'Aragon',
    'ES-AS': u'Asturias',
    'ES-CB': u'Cantabria',
    'ES-CM': u'Castile-La Mancha',
    'ES-CL': u'Castile and León',
    'ES-CT': u'Catalonia',
    'ES-MD': u'Community of Madrid',
    'ES-VC': u'Valencian Community',
    'ES-EX': u'Extremadura',
    'ES-IB': u'Balearic Islands',
    'ES-CN': u'Canary Islands',
    'ES-GA': u'Galicia',
    'ES-RI': u'La Rioja',
    'ES-NC': u'Navarre',
    'ES-MC': u'Region of Murcia',
    'ES-CE': u'Ceuta',
    'ES-ML': u'Melilla',
    'ES-PV': u'Basque Country',
    'ES-MAGRAMA': u'MAGRAMA'
}

In [51]:
pb.output('Retrieving --> WLE SCI list')
sci_list_page = pb.Page(commons_site, SCI_DB_PAGE)
sci_list_text = StringIO(sci_list_page.text[sci_list_page.text.find('\n') + 
                                                      1:sci_list_page.text.rfind('\n')])
sci_df = pd.read_csv(sci_list_text, sep=";",
                     index_col=False,
                     names=["name", "code", "magrama_url", "community",
                            "bio_region", "continent", "min_altitude",
                            "max_altitude", "avg_altitude", "longitude",
                            "latitude", "area", "marine_percentage",
                            "marine_area", "image", "commons_cat", "wikidata_id"])
pb.output('Retrieved --> WLE SCI list')
codes = sci_df["code"].tolist()

Retrieving --> WLE SCI list
Retrieved --> WLE SCI list


In [52]:
cat_wle = pb.Category(commons_site, WLE_CATEGORY)
gen_wle = pagegenerators.CategorizedPageGenerator(cat_wle)

images_wle = [page.title(withNamespace=True) for page in gen_wle if page.is_filepage()]
#len(images_wle)

In [53]:
list_page = pb.Page(commons_site, LOG_PAGE)

In [57]:
list_text = u'<pre>\n'
image_counter = 0
for image in images_wle:
    page = pb.Page(commons_site, image)
    text = page.text
    wikicode = mwh.parse(text)
    templates = wikicode.filter_templates()

    if (image_counter != 0) and (image_counter % 50 == 0) :
        pb.output ('Retrieving --> %d image descriptions downloaded' %(image_counter))
    image_counter += 1
    image_item = [None] * 10
    image_item[0] = page.title(withNamespace=False) # title
    creation = page.oldest_revision
    print image
    if creation["user"] != u'WikiLovesESBot' :
        image_item[1] = creation["user"]                # uploader
        try:
            # Too old users do not have a user registration time
            user_registration = pb.User(commons_site, title=creation["user"]).registration()
            image_item[2] = str((start_datetime - user_registration).days) # days from registration
        except :
            image_item[2] = "3650"
    else :
        info_template = [template for template in wikicode.filter_templates() 
                        if template.name.lower().strip() == u"information"][0]
        image_item[1] = re.findall(r"\[(.*)\]", info_template.get("Author").value.strip())[0]
        image_item[2] = "-1"
        
    image_item[3] = (creation.timestamp + timedelta(hours=2)).strftime("%Y-%m-%d %H:%M:%S") # creation time

    WLE_template_found = False
    WLE_identifier = ''
    lic_templates = [template for template in wikicode.filter_templates() 
                    if template.name.lower().strip() == u"lic"]
    if len(lic_templates) > 0:
        WLE_template_found = True
        WLE_identifier = lic_templates[0].get(1).value.strip()
        
    if WLE_identifier in codes :
        image_item[4] = WLE_identifier
        sci_name = sci_df[sci_df["code"] == WLE_identifier]["name"].iloc[0]
        image_item[5] = unicode(sci_name, "utf-8", "replace")
        community_id  = sci_df[sci_df["code"] == WLE_identifier]["community"].iloc[0]
        image_item[6] = communities[community_id]
        if sci_df[sci_df["code"] == WLE_identifier]["commons_cat"].iloc[0] is not np.nan :
            image_item[7] = unicode(sci_df[sci_df["code"] == WLE_identifier]["commons_cat"].iloc[0], "utf-8", "replace")
        else :
            image_item[7] = ''
        image_item[8] = str(sci_df[sci_df["code"] == WLE_identifier]["latitude"].iloc[0])
        image_item[9] = str(sci_df[sci_df["code"] == WLE_identifier]["longitude"].iloc[0])
    else :
        image_item[4] = ''
        image_item[5] = ''
        image_item[6] = ''
        image_item[7] = ''
        image_item[8] = ''
        image_item[9] = ''

    row_text = u'%s;\n' % (';'.join(image_item))
    list_text += row_text

list_text += u'</pre>'

File:"Encina andante". Parque Natural de la Subbética Cordobesa. Cabra.JPG
File:'Grasilla' (Pinguicula grandiflora) en la Senda del Arcediano. Amieva (Asturias). Parque Nacional Picos de Europa. ES1200001. ROSUROB.JPG
File:(1) ABEJA LIBANDO FLOR DEL NARANJO EN EL CORRAL DE "LA CASINA" EN GATA (CACERES).JPG
File:(2) ABEJA LIBANDO FLOR DEL NARANJO EN EL CORRAL DE "LA CASINA" EN GATA (CACERES).JPG
File:0004395970-original.jpg
File:0004396045-original.jpg
File:0004396096-original.jpg
File:0004396229-original.jpg
File:0004396610-original.jpg
File:0004451849-original.jpg
File:0004451867-original.jpg
File:0004451877-original.jpg
File:0004451886-original.jpg
File:002 Castell de Montsoriu (Arbúcies-Sant Feliu de Boxareu9 Montseny ES5110001.jpg
File:002 castell Montsoriu (Arbúcies- Sant Feliu de Buixareu) Montseny ES 5110001.jpg
File:01 Hornachos.JPG
File:010 Garza imperial al Parc Natural dels Aiguamolls de l'Empordà ES0000019.jpg
File:02 Hornachos.JPG
File:02042010 125945 56032 - WLE Spain 201

Retrieving --> 50 image descriptions downloaded


File:02042010 161121 56221 - WLE Spain 2015.jpg
File:02042010 161323 56223 - WLE Spain 2015.jpg
File:02042010 162049 56225 - WLE Spain 2015.jpg
File:02042010 162728 56227 - WLE Spain 2015.jpg
File:02042010 162740 56228 - WLE Spain 2015.jpg
File:02042010 163420 56231 - WLE Spain 2015.jpg
File:02042010 163720 56236 - WLE Spain 2015.jpg
File:02042010 163810 56238 - WLE Spain 2015.jpg
File:02042010 163856 56239 - WLE Spain 2015.jpg
File:02042010 175210 55611 - WLE Spain 2015.jpg
File:02042010 175210 56243 - WLE Spain 2015.jpg
File:03 Hornachos.JPG
File:04 Hornachos.JPG
File:05 Hornachos.JPG
File:05062010 120542 58167 - WLE Spain 2015.jpg
File:05062010 123320 58216 - WLE Spain 2015.jpg
File:05062010 123403 58217 - WLE Spain 2015.jpg
File:06062004 141417 4361 - WLE Spain 2015.jpg
File:06062004 141424 4362 - WLE Spain 2015.jpg
File:06062004 141919 4365 - WLE Spain 2015.jpg
File:06062004 142334 4368 - WLE Spain 2015.jpg
File:06062004 142841 4375 - WLE Spain 2015.jpg
File:06062004 143018 4376 -

Retrieving --> 100 image descriptions downloaded


File:09042009 165043 CEC 0099 - WLE Spain 2015.jpg
File:09042009 165453 CEC 0103 - WLE Spain 2015.jpg
File:09042009 165526 CEC 0104 - WLE Spain 2015.jpg
File:09042009 165711 CEC 0108 - WLE Spain 2015.jpg
File:09042009 165804 CEC 0109 - WLE Spain 2015.jpg
File:09042009 170048 CEC 0112 - WLE Spain 2015.jpg
File:09042009 180112 CEC 0145 - WLE Spain 2015.jpg
File:09042009 181434 CEC 0153 - WLE Spain 2015.jpg
File:0olga lesiv.JPG
File:10olga lesiv.JPG
File:11042015-DSC 8580.jpg
File:11112012 090226 4321 - WLE Spain 2015.jpg
File:11112012 090226 4322 - WLE Spain 2015.jpg
File:11112012 090226 4323 - WLE Spain 2015.jpg
File:11112012 090226 4325 - WLE Spain 2015.jpg
File:11112012 090228 4334 - WLE Spain 2015.jpg
File:12-52 capture the season - WLE Spain 2015.jpg
File:12olga lesiv.JPG
File:14. IMG 2213 copy copy.jpg
File:14. IMG 2215.JPG
File:14. IMG 2216.JPG
File:16. IMG 2220.JPG
File:16. IMG 2222.JPG
File:16082011 110052 PIR SEL 1128 - WLE Spain 2015.jpg
File:16c. P1020615.JPG
File:17. IMG 153

...

File:Vista de Tossa de Mar i la Església vella de Sant Vicenç.JPG
File:Vista del Bembezar.jpg
File:Vista del El Teide.jpg
File:Vista del Parque Natural de Sierra Nevada.JPG
File:Vista del Rio Aragon a su paso por el Puente medieval los Rocaleses en Yesa.JPG
File:Vista del río Tajo.JPG
File:Vista del valle de Valdivielso hacia el norte.jpg
File:Vista desde Cueva Rosa.JPG
File:Vista desde el Sendero de los Cazadores - WLE Spain 2015 (1).jpg
File:Vista desde el Sendero de los Cazadores - WLE Spain 2015.jpg
File:Vista desde embalse en castillo de Peñaroya.jpg
File:Vista desdeTrevejo.JPG
File:Vista general de Fataga y el Barranco de Tirajana.jpg
File:Vista General del Rio Irati en La Foz de Lumbier.JPG
File:Vista general dels cingles y el el monasteri de Sant Miquel del Fai.JPG
File:Vista hacia las Hoces del Río Duratón.JPG
File:Vista hacia Las Palmas.jpg
File:Vista occidental.JPG
File:Vista Oriental.JPG
File:Vista panorámica de la playa de Punta Brava.JPG
File:Vista parcial del Lago Ercina

Retrieving --> 4950 image descriptions downloaded


File:Viñedos Ecológicos en la Sierra de Salinas, Yecla (Murcia).jpg
File:Viñedos sobre el Sil - WLE Spain 2015.jpg
File:Volar en Larouco.JPG
File:Volcán de la Arena - WLE Spain 2015.jpg
File:Volcán en Lanzarote.jpg
File:Vuelo parapente.JPG
File:Vélez Blanco.JPG
File:Vía láctea desde Valdavido.jpg
File:Vía Nova - WLE Spain 2015.jpg
File:Vía Nova, tramo restaurado - WLE Spain 2015.jpg
File:Vía Verde Beni-Orpesa02.JPG
File:Vía Verde Beni-Orpesa08.JPG
File:Vía Verde Beni-Orpesa10.JPG
File:Vía Verde Beni-Orpesa13.JPG
File:Vía Verde Beni-Orpesa14.JPG
File:Vía Verde Beni-Orpesa27.JPG
File:Vía Verde Beni-Orpesa32.JPG
File:WALKING.JPG
File:Week 59-104, After the storm - WLE Spain 2015.jpg
File:WIKI 559 MmoramoraleS.JPG
File:WIKI 560 MmoramoraleS.JPG
File:WIKI 562 MmoramoraleS.JPG
File:WIKI 563 MmoramoraleS.JPG
File:WIKI 564 MmoramoraleS.JPG
File:WIKI 565 MmoramoraleS.JPG
File:WIKI 566 MmoramoraleS.JPG
File:WIKI 567 MmoramoraleS.JPG
File:WIKI 568 MmoramoraleS.JPG
File:WIKI 569 MmoramoraleS.JPG
F

Retrieving --> 5000 image descriptions downloaded


File:Zona litoral Vallcarca.JPG
File:Zona recreativa,Fuente del Estudiante.JPG
File:Zorita del Maestrazgo 01.JPG
File:Zorita del Maestrazgo 02.JPG
File:Zorita del Maestrazgo 03.JPG
File:Zorro cazorla.jpg
File:Zoñar.jpg
File:Zuheros, desde el Cañón del Bailón.jpg
File:Zuriza - WLE Spain 2015 (1).jpg
File:Zuriza - WLE Spain 2015 (2).jpg
File:Zuriza - WLE Spain 2015 (3).jpg
File:Zuriza - WLE Spain 2015 (4).jpg
File:Zuriza - WLE Spain 2015 (5).jpg
File:Zuriza - WLE Spain 2015 (6).jpg
File:Zuriza - WLE Spain 2015 (7).jpg
File:Zuriza - WLE Spain 2015 (8).jpg
File:Zuriza - WLE Spain 2015 (9).jpg
File:Zuriza - WLE Spain 2015.jpg
File:Ànec blanc o ànnera blanca (Tadorna tadorna).jpg
File:Árbol Garoé.JPG
File:Árboles en las hoces.JPG
File:Árbore Olveira.jpg
File:Área de esquiar en el Parque Natural de Sierra Nevada.JPG
File:Área Recreativa da Cortevella.JPG


In [58]:
list_page.text = list_text
pb.output('Publishing --> WLE {0} image log'.format(YEAR))
list_page.save(u"WLE Spain {0} log".format(YEAR))

Publishing --> WLE 2015 image log
Sleeping for 18.2 seconds, 2017-06-14 00:40:26
Page [[Commons:Wiki Loves Earth 2015 in Spain/Log]] saved
